# Align bible sentences
This script deserves a better treatment.

In [1]:
import pandas as pd
import numpy as np

vers = ["nirv", "tot11", "ntb", "ctb"]
datas = {}

import re
def is_verse(ind):
    return bool(re.match("^\d+:\d+$", ind.split(" ")[1]))
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]
def get_books(data):
    return f7(ind.split(" ")[0] for ind in data.index)

for ver in vers:
    data = pd.read_csv('bible_sentences_' + ver + '.csv', index_col="index")
    data.columns = [[ver, ver + "_source"]]
    data[ver + "_order"] = np.arange(len(data))
    datas[ver] = data
    n_verses = sum(is_verse(ind) for ind in data.index)
    books = get_books(data)
    print("Version: %s, sentences: %d, verses: %d, books: (%d) \n\t%s\n" % (ver, len(data), n_verses, len(books), "|".join(books)))
    

Version: nirv, sentences: 33969, verses: 31087, books: (66) 
	GEN|EXO|LEV|NUM|DEU|JOS|JDG|RUT|1SA|2SA|1KI|2KI|1CH|2CH|EZR|NEH|EST|JOB|PSA|PRO|ECC|SNG|ISA|JER|LAM|EZK|DAN|HOS|JOL|AMO|OBA|JON|MIC|NAM|HAB|ZEP|HAG|ZEC|MAL|MAT|MRK|LUK|JHN|ACT|ROM|1CO|2CO|GAL|EPH|PHP|COL|1TH|2TH|1TI|2TI|TIT|PHM|HEB|JAS|1PE|2PE|1JN|2JN|3JN|JUD|REV

Version: tot11, sentences: 25039, verses: 22224, books: (39) 
	GEN|EXO|LEV|NUM|DEU|JOS|JDG|RUT|1SA|2SA|1KI|2KI|1CH|2CH|EZR|NEH|EST|JOB|PSA|PRO|ECC|SNG|ISA|JER|LAM|EZK|DAN|HOS|JOL|AMO|OBA|JON|MIC|NAM|HAB|ZEP|HAG|ZEC|MAL

Version: ntb, sentences: 9847, verses: 8232, books: (30) 
	RUT|EST|JON|MAT|MRK|LUK|JHN|ACT|ROM|1CO|2CO|GAL|EPH|PHP|COL|1TH|2TH|1TI|2TI|TIT|PHM|HEB|JAS|1PE|2PE|1JN|2JN|3JN|JUD|REV

Version: ctb, sentences: 9736, verses: 8099, books: (28) 
	GEN|MAT|MRK|LUK|JHN|ACT|ROM|1CO|2CO|GAL|EPH|PHP|COL|1TH|2TH|1TI|2TI|TIT|PHM|HEB|JAS|1PE|2PE|1JN|2JN|3JN|JUD|REV



In [196]:
# 1 build an index. considering order
# 2 put into one DF


In [2]:
all_books = 'GEN|EXO|LEV|NUM|DEU|JOS|JDG|RUT|1SA|2SA|1KI|2KI|1CH|2CH|EZR|NEH|EST|JOB|PSA|PRO|ECC|SNG|ISA|JER|LAM|EZK|DAN|HOS|JOL|AMO|OBA|JON|MIC|NAM|HAB|ZEP|HAG|ZEC|MAL|MAT|MRK|LUK|JHN|ACT|ROM|1CO|2CO|GAL|EPH|PHP|COL|1TH|2TH|1TI|2TI|TIT|PHM|HEB|JAS|1PE|2PE|1JN|2JN|3JN|JUD|REV'.split("|")
book_order = {book:i for i,book in enumerate(all_books)}


In [3]:
def parse_index(ind):
    book_str, tmp = ind.split(" ")
    chap_str, v_str = tmp.split(":")
    return book_str, chap_str, v_str
# def cmp_fn(a, b):
#     (a_book, a_chap, a_v), (b_book, b_chap, b_v) = parse_index(a), parse_index(b)
#     if book_order[a_book] < book_order[b_book]: return 1
#     if book_order[a_book] > book_order[b_book]: return -1
#     if not is_verse(a) and is_verse(b): return 1
#     if is_verse(a) and not is_verse(b): return -1
#     if not is_verse(a) and not is_verse(b): 
#         if a < b: return 1
#         if a > b: return -1
#         return 0
#     if (int(a_chap), int(a_v)) <  (int(b_chap), int(b_v)): return 1
#     if (int(a_chap), int(a_v)) >  (int(b_chap), int(b_v)): return -1
#     return 0

In [4]:
def key_fn(ind):
    book, chap, v = parse_index(ind)
    def make_key(s):
        if s.isnumeric():
            return "%03d"%int(s)
        return '-'+s
    return book_order[book], make_key(chap), make_key(v)
    

In [5]:
all_verse_index = []
for ver in vers:
    all_verse_index += [ind for ind in datas[ver].index if is_verse(ind)]
all_verse_index = sorted(list(set(all_verse_index)), key = key_fn)

columns = []
for ver in vers:
    columns += [ver, ver+'_source']
columns

['nirv',
 'nirv_source',
 'tot11',
 'tot11_source',
 'ntb',
 'ntb_source',
 'ctb',
 'ctb_source']

In [6]:
len(all_verse_index)

31104

In [7]:
combined = pd.DataFrame(index=all_verse_index, columns=columns)

In [8]:
for ver in vers:
    for ind in all_verse_index:
        if ind in datas[ver].index:
            combined.at[ind, ver] = datas[ver].loc[ind, ver].values[0]
            combined.at[ind, ver+"_source"] = datas[ver].loc[ind, ver+"_source"].values[0]
    print("Done Processing..." , ver)

Done Processing... nirv
Done Processing... tot11
Done Processing... ntb
Done Processing... ctb


In [9]:
combined

,nirv,nirv_source,tot11,tot11_source,ntb,ntb_source,ctb,ctb_source
GEN 1:1,"In the beginning, God created the heavens and ...",https://www.bible.com/bible/110/GEN.1.NIRV,ཐོག་མར་དཀོན་མཆོག་གིས་ནམ་མཁའ་དང་འཛམ་གླིང་བཀོད་དུས།,https://www.bible.com/bible/1712/GEN.1.TOT11,NaN,NaN,ཐོག་མར་དཀོན་མཆོག་གིས་འཇིག་རྟེན་ཁམས་དང་དེར་ཡོད་...,https://www.gsungrab.org/online_tibetan_bible/...
GEN 1:2,The earth didn’t have any shape. And it was em...,https://www.bible.com/bible/110/GEN.1.NIRV,ས་ནི་དབྱིབས་མེད་སྟོང་པ་ཡོད་པ་དང་། ས་གནས་གང་སར་...,https://www.bible.com/bible/1712/GEN.1.TOT11,NaN,NaN,ཡང་ས་ནི་དབྱིབས་མེད་སྟོང་པ་ཡིན་པ་དང༌། རྒྱ་མཚོ་...,https://www.gsungrab.org/online_tibetan_bible/...
GEN 1:3,"God said, “Let there be light.” And there was ...",https://www.bible.com/bible/110/GEN.1.NIRV,དཀོན་མཆོག་གིས་འོད་བྱུང་ཞིག་ཅེས་བཀའ་གནང་བ་དང་། ...,https://www.bible.com/bible/1712/GEN.1.TOT11,NaN,NaN,དེ་ཡང་དཀོན་མཆོག་གིས་“འོད་འབྱུང་ཞིག་”ཅེས་གསུངས་...,https://www.gsungrab.org/online_tibetan_bible/...
GEN 1:4,God saw that the light was good. He separated ...,https://www.bible.com/bible/110/GEN.1.NIRV,དཀོན་མཆོག་གིས་འོད་ལེགས་པ་ཡིན་པར་གཟིགས་ནས་ཐུགས་...,https://www.bible.com/bible/1712/GEN.1.TOT11,NaN,NaN,དཀོན་མཆོག་གིས་འོད་དེ་ལེགས་པ་ཡིན་པར་གཟིགས་ཏེ་མུ...,https://www.gsungrab.org/online_tibetan_bible/...
GEN 1:5,God called the light “day.” He called the dark...,https://www.bible.com/bible/110/GEN.1.NIRV,དཀོན་མཆོག་གིས་འོད་ཀྱི་མིང་ལ་ཉིན་མོ་དང་། མུན་པའ...,https://www.bible.com/bible/1712/GEN.1.TOT11,NaN,NaN,དཀོན་མཆོག་གིས་འོད་ཀྱི་མིང་ལ་ཉིན་མོ་ཞེས་བཏགས། ...,https://www.gsungrab.org/online_tibetan_bible/...
...,...,...,...,...,...,...,...,...
REV 22:17,"The Holy Spirit and the bride say, “Come!” And...",https://www.bible.com/bible/110/REV.22.NIRV,NaN,NaN,དམ་པའི་ཐུགས་ཉིད་དང་བག་གསར་གྱིས་ཕེབས་ཤོག་དང་ཞེས...,https://www.bible.com/bible/1933/REV.22.NTB,དམ་པའི་ཐུགས་ཉིད་དང་མནའ་མ་ཡིས་“ཕེབས་ཤོག་”ཅེས་གས...,https://www.bible.com/bible/2378/REV.22.CTB
REV 22:18,I am warning everyone who hears the words of t...,https://www.bible.com/bible/110/REV.22.NIRV,NaN,NaN,བདག་གིས་མདོ་འདིའི་ལུང་བསྟན་ཐོས་མཁན་ཐམས་ཅད་ལ་བད...,https://www.bible.com/bible/1933/REV.22.NTB,མདོ་འདིའི་ལུང་བསྟན་གྱི་བཀའ་ཐོས་མཁན་ཐམས་ཅད་ལ་ངས...,https://www.bible.com/bible/2378/REV.22.CTB
REV 22:19,Suppose someone takes away any words from this...,https://www.bible.com/bible/110/REV.22.NIRV,NaN,NaN,ཡང་སུ་ཞིག་གིས་མདོ་འདིའི་ནང་གི་ལུང་བསྟན་ལས་གང་ཞ...,https://www.bible.com/bible/1933/REV.22.NTB,ཡང་སུ་ཞིག་གིས་མདོ་འདི་ལ་བྲིས་པའི་ལུང་བསྟན་གྱི་...,https://www.bible.com/bible/2378/REV.22.CTB
REV 22:20,Jesus is a witness about these things. He says...,https://www.bible.com/bible/110/REV.22.NIRV,NaN,NaN,དོན་འདི་རྣམས་ལ་བདེན་དཔང་མཛད་མཁན་གྱིས་འདི་སྐད་ད...,https://www.bible.com/bible/1933/REV.22.NTB,དོན་འདི་རྣམས་ལ་དཔང་པོ་བྱེད་མཁན་གྱིས་“ལགས་ཡིན། ...,https://www.bible.com/bible/2378/REV.22.CTB


In [10]:
combined.to_csv("bible_verse_only.csv")

In [11]:
combined.sample(20).to_csv("bible_verse_only_sample.csv")